In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [161]:
#Define functions that identify each variable that we will be using. 
#Date function will be indexed after all season data is present.

#Define Player Name function
def PName(matches):
    PName = []
    for leg in matches:
        if leg[0][0] == 0:
            PName.append(leg[0][1])
        else:
            PName.append(leg[0][2])
    return PName

#Define Leg Code function
def LegCode(matches):
    LegCode = []
    for leg in matches:
        if leg[0][0] != 0:
            thisleg = leg[0][0]
            LegCode.append(leg[0][0])
        else:
            LegCode.append(thisleg)
    return LegCode

#Define Game Name function
def Game(matches):
    Game = []
    for leg in matches:
        if leg[0][0] != 0:
            thisgame = leg[0][1]
            Game.append(leg[0][1])
        else:
            Game.append(thisgame)
    return Game

#Define Player 3 Dart Average function
def P3DA(matches):
    P3DA = []
    for leg in matches:
        if leg[0][0] != 0:
            P3DA.append(leg[0][5])
        elif (leg[0][0] == 0) & (leg[0][4] == 0):
            P3DA.append(leg[0][2])
        else:
            P3DA.append(leg[0][4])
    for avg in P3DA:
        #301 legs with a zero average sometimes return cork value
        if (avg == 'C') or (avg == 'S'):
            avg = 0
    return P3DA

#Define Leg Win Function
def LegWin(matches):
    LegWin = []
    for leg in matches:
        if leg[0][0] != 0:
            LegWin.append(leg[0][9])
        elif (leg[0][0] == 0) & (leg[0][4] == 0):
            LegWin.append(leg[0][6])
        else:
            LegWin.append(leg[0][8])
    return LegWin

#Define darts thrown function
def dartsthrown(matches):
    dt = []
    for leg in matches:
        dt.append(leg[0][-7])
    return dt

#Define points/marks function
def ptsmks(matches):
    scores = []
    for leg in matches:
        if leg[0][-5] != 0:
            mk = leg[0][-5].replace("M","")
            scores.append(int(mk))
        else:
            scores.append(int(leg[0][-6]))
    return scores

#Define turn sequence function
def turnseq(matches):
    turns = []
    for leg in matches:
        try:
            turns.append(leg[0][-14])
        except IndexError:
            turns.append('?')
    return turns


In [7]:
#Leg set function, fill zeroes for blanks
def leggify(x):
    gmst = []
    for i in x:
        if (i.text == '') or (i.text == '\xa0'):
            gmst.append(0)
        else:
            gmst.append(i.get_text())
    return gmst

In [190]:
#Define the function that takes the ID of a match and then derives the game table 

def MatchTable(match):
    mitch = str(match)
    matchloc = 'https://members.dartconnect.com/history/report/players/match/' + str(match)
    summaryloc = 'https://members.dartconnect.com/history/report/match/' + str(match)
    matchpage = requests.get(matchloc)
    summarypage = requests.get(summaryloc)
    soup = BeautifulSoup(matchpage.text, 'lxml')
    psoup = BeautifulSoup(summarypage.text, 'lxml')
    
    #Find and set the date string
    datematch = soup.h2.next_element.next_element
    daydate = datematch.td.text
    day = daydate[0:3]
    pdate = daydate[-8:]
    if (pdate[0] == ","):
        date = pdate[1:]
    else:
        date = pdate
        
    #Identify game detail table
    game_pp = soup.find(class_="freeTable playerPref")    
    
    #Break table into rows
    rows = [row for row in game_pp.find_all("tr")]
    
    #Call leg set function for all legs
    all_legs = []
    for row in rows[1:]:
        rowset = row.find_all('td')
        pleg = [leggify(rowset)]
        all_legs.append(pleg)

    #Remove match separator rows
    for leg in all_legs:
        if (leg[0][1] == 0):
            all_legs.remove(leg)
    
    #Team roster and team name definitions
    teams = psoup.find_all(class_="playerNameHeader")
    team1 = [teams[0].text]
    team2 = [teams[1].text]
    tm1nm = teams[0].text
    tm2nm = teams[1].text

    #Refer to table data on summary page
    homaw = psoup.find_all('td', text=True)

    #Get last 40 and define excluded strings
    homaw = homaw[-40::]
    forbid = ['Premium', 'Recap Sent','Missing Email','Member','&nbsp']

    #Team 2 roster construct
    for pfind in homaw[::-1]:
        row = pfind.text
        if row in forbid:
            continue
        elif row == "Away/Guest Players":
            break
        else:
            team2.append(row)
            continue

    #Team 1 roster construct        
    for pfind in homaw[::-1]:
        row = pfind.text
        if (row in team2) or (row in forbid) or (row == 'Away/Guest Players'):
            continue
        elif row == "Home/Host Players":
            break
        else:
            team1.append(row)

    #Redundancy check
    for player in team1:
        if player in team2:
            print(player, "ON OTHER TEAM")

    #Add aliases to roster

    #Find rows populated with player names in summary table
    players = psoup.find_all('tr')
    versus = []
    for row in players:
        opps = row.find_all(class_='playerName')
        if (opps != []):
            versus.append(opps)
        else:
            continue

    #Differentiate between solo and partner matches, then divide players into teams
    t1 = []
    t2 = []
    positcol = []
    for match in versus:
        if (len(match) == 2):
            t1.append(match[0].text)
            positcol.append('1')

            t2.append(match[1].text)
            positcol.append('2')

        elif (len(match) == 4):
            t1.append(match[0].text)
            positcol.append('1')

            t1.append(match[1].text)
            positcol.append('2')

            t2.append(match[2].text)
            positcol.append('3')

            t2.append(match[3].text)
            positcol.append('4')

        else:
            print("I don't know who's playing this match.")

    #Generates tm1 and tm2 spaceless lists
    tm1 = []
    tm2 = []

    #Spaceless from rosters
    for p in team1:
        p = "".join(p.split())
        tm1.append(p)

    for p in team2:
        p = "".join(p.split())
        tm2.append(p)

    #Spaceless aliases from summary table
    for t in t1:
        t = "".join(t.split())
        if t in tm1:
            continue
        else:
            tm1.append(t)

    for t in t2:
        t = "".join(t.split())
        if t in tm2:
            continue
        else:
            tm2.append(t)

    #Generate spaceless player list from PName function
    all_players = PName(all_legs)
    plyrlst = []

    for playr in all_players:
        playr = "".join(playr.split())
        plyrlst.append(playr)

    #Generates the 'Team' column for the dataframe
    teamcol = []
    homecol = []
    for player in plyrlst:
        if player in tm1:
            teamcol.append(tm1nm)
            homecol.append('Home')
        elif player in tm2:
            teamcol.append(tm2nm)
            homecol.append('Away')
        else:
            teamcol.append('Mysterious Ronin')
            homecol.append('Ronin')
            print(player, "not found on any team.")
            
    #Construct game table for match
    gametable = pd.DataFrame(
            {
                "Team": teamcol,
                "Date": date,
                "Home/Away": homecol,
                "Leg_Code": LegCode(all_legs),
                "Game": Game(all_legs),
                "3DA": pd.Series(P3DA(all_legs)),
                "Player": PName(all_legs),
                "W/A": LegWin(all_legs),
                "Darts_Thrown": dartsthrown(all_legs),
                "Points/Marks": ptsmks(all_legs),
                "Turn_Sequence": turnseq(all_legs),
                "Match": mitch
            })
    
    return gametable

In [49]:
matches = pd.read_csv("Data/WCD-20W-Matches.csv")

In [30]:
matches

,Unnamed: 0,0
0,0,5e1e73504a6ca334944cddae
1,1,5e27aec32cba526823fc6a55
2,2,5e30e2644e062202124a9e84
3,3,5e3a2528b274ba05c62721d8
4,4,5e435cd9053d5933ff6a5d36
...,...,...
186,186,5e435b72053d5933ff6a5b77
187,187,5e30ea924e062202124aa7c7
188,188,5e435ff5053d5933ff6a610c
189,189,5e1e76604a6ca334944ce09c


In [169]:
m = MatchTable('5e1e73504a6ca334944cddae')

In [128]:
def dartsthrown(matches):
    dt = []
    for leg in matches:
        dt.append(leg[0][-7])
    return dt

In [159]:
def turnseq(matches):
    turns = []
    for leg in matches:
        try:
            turns.append(leg[0][-14])
        except IndexError:
            turns.append('?')
    return turns

In [155]:
def ptsmks(matches):
    scores = []
    for leg in matches:
        if leg[0][-5] != 0:
            mk = leg[0][-5].replace("M","")
            scores.append(int(mk))
        else:
            scores.append(int(leg[0][-6]))
    return scores

## Start of Season Scraping Activity

In [241]:
#Input season match data
seasonmatch = pd.read_csv('Data/WCD-16S-Matches.csv')
seasonmatch = seasonmatch['0']

In [242]:

#Get rid of matches not held on official league pages
Season_Matches = []
for match in seasonmatch:
    if (len(match) > 9):
        Season_Matches.append(match)

#Assemble all season legs
Season_Legs = []
for match in Season_Matches:
    Season_Legs.append(MatchTable(match))

#Concatenate into a dataframe
table = pd.concat(Season_Legs)
table.reset_index()

,index,Team,Date,Home/Away,Leg_Code,Game,3DA,Player,W/A,Darts_Thrown,Points/Marks,Turn_Sequence,Match
0,0,Bacon,6-7-16,Home,1.1,501 SIDO,24.6,Jason Callan,W,61,501,1,57577da1c3cd6f0f304014b0
1,1,Motley Crew,6-7-16,Away,1.1,501 SIDO,24.4,Ken Molitoris,0,60,487,2,57577da1c3cd6f0f304014b0
2,2,Bacon,6-7-16,Home,1.2,501 SIDO,32.0,Jason Callan,W,47,501,2,57577da1c3cd6f0f304014b0
3,3,Motley Crew,6-7-16,Away,1.2,501 SIDO,30.8,Ken Molitoris,0,48,493,1,57577da1c3cd6f0f304014b0
4,4,Bacon,6-7-16,Home,2.1,301 DIDO,31.4,Kurt Nishi,A,15,157,2,57577da1c3cd6f0f304014b0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5618,67,Dart Team,6-21-16,Away,9.2,501 SIDO,58.5,Kevin Kruse,0,12,234,4,5769eb6d4466ed176e203392
5619,68,The Thatch Pub,6-21-16,Home,9.3,501 SIDO,67.1,Brian DiCrescenzo,W,16,358,1,5769eb6d4466ed176e203392
5620,69,The Thatch Pub,6-21-16,Home,9.3,501 SIDO,28.6,Anne Quinn,A,15,143,3,5769eb6d4466ed176e203392
5621,70,Dart Team,6-21-16,Away,9.3,501 SIDO,62.6,Mark Kokot,0,15,313,2,5769eb6d4466ed176e203392


In [243]:
table['Season'] = 2016.2

In [244]:
table.head()

,Team,Date,Home/Away,Leg_Code,Game,3DA,Player,W/A,Darts_Thrown,Points/Marks,Turn_Sequence,Match,Season
0,Bacon,6-7-16,Home,1.1,501 SIDO,24.6,Jason Callan,W,61,501,1,57577da1c3cd6f0f304014b0,2016.2
1,Motley Crew,6-7-16,Away,1.1,501 SIDO,24.4,Ken Molitoris,0,60,487,2,57577da1c3cd6f0f304014b0,2016.2
2,Bacon,6-7-16,Home,1.2,501 SIDO,32.0,Jason Callan,W,47,501,2,57577da1c3cd6f0f304014b0,2016.2
3,Motley Crew,6-7-16,Away,1.2,501 SIDO,30.8,Ken Molitoris,0,48,493,1,57577da1c3cd6f0f304014b0,2016.2
4,Bacon,6-7-16,Home,2.1,301 DIDO,31.4,Kurt Nishi,A,15,157,2,57577da1c3cd6f0f304014b0,2016.2


In [245]:
table.to_csv('WCD2016-2.csv')

## Start of Derive Features

In [246]:
f19 = pd.read_csv("WCD2019-3.csv")
w20 = pd.read_csv("WCD2020-1.csv")

In [248]:
f19['TDA'] = (f19['Points/Marks'] / f19['Darts_Thrown']) * 3

In [254]:
f19['Win'] = (f19['W/A'] != '0')

In [348]:
f19['Set'] = pd.array(np.floor(f19['Leg_Code']), dtype=pd.Int64Dtype())

In [372]:
#Problems with this Series. Had different values for what appeared to be the same. Round fixed.
f19['Leg'] = pd.array(round((f19['Leg_Code']-f19['Set'])*10), dtype=pd.Int64Dtype())

In [362]:
f19['Cork/Start'] = (f19['Turn_Sequence'] == 1) | (f19['Turn_Sequence'] == 3)

In [335]:
f19['Home'] = f19['Home/Away'] == 'Home'

In [373]:
pd.value_counts(f19['Leg'])

1    6360
2    6360
3    3076
Name: Leg, dtype: Int64

In [375]:
f19.infer_objects().dtypes

Unnamed: 0         int64
Team              object
Date              object
Home/Away         object
Leg_Code         float64
Game              object
3DA               object
Player            object
W/A               object
Darts_Thrown       int64
Points/Marks       int64
Turn_Sequence      int64
Match             object
Season           float64
TDA              float64
Win                 bool
Set                Int64
Leg                Int64
Cork                bool
Cork/Start          bool
Home                bool
dtype: object

In [391]:
lu = f19.groupby(['Season','Team','Set','Game'])['TDA'].mean().

In [392]:
lu.head(24)

Season  Team         Set  Game    
2019.3  140 Darts    1    501 SIDO    45.317739
                     2    301 DIDO    29.486012
                          501 SIDO    46.392152
                          Cricket      2.133496
                     3    301 DIDO    40.860907
                          Cricket      2.901629
                     4    301 DIDO    36.625000
                          501 SIDO    46.571429
                          Cricket      2.070106
                     5    301 DIDO    31.998768
                          501 SIDO    59.156143
                          Cricket      2.616440
                     6    301 DIDO    36.905723
                          501 SIDO    49.850890
                          Cricket      1.994175
                     7    301 DIDO    32.079474
                     8    Cricket      2.086859
                     9    501 SIDO    48.117386
        Adjustments  1    501 SIDO    30.742334
                     2    301 DIDO    26.900116
     